# NLP Toxcicty Machine Learning Model

## Imports

In [25]:
import pandas as pd
import string

from nltk.corpus import stopwords, wordnet
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.multioutput import MultiOutputClassifier

import re

## Data

In [26]:
df_train= pd.read_csv('data/train.csv', index_col = 'id')

In [27]:
df_train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [28]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159571 entries, 0000997932d777bf to fff46fc426af1f9a
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   comment_text   159571 non-null  object
 1   toxic          159571 non-null  int64 
 2   severe_toxic   159571 non-null  int64 
 3   obscene        159571 non-null  int64 
 4   threat         159571 non-null  int64 
 5   insult         159571 non-null  int64 
 6   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 9.7+ MB


In [29]:
df_test = pd.read_csv('data/test_labels.csv')

In [30]:
df_test.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [31]:
# Function for improving parts of speech information

### get_wordnet_pos was taken from Lecture 51-nlp_modeling.ipynb 
### link to the lecture: https://github.com/dvdhartsman/NTL-DS-080723/blob/main/4phase/51-nlp_modeling.ipynb


def get_wordnet_pos(treebank_tag):
    '''
    Translate nltk POS to wordnet tags
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [32]:
# Function for handling the transformation of data

### preprocess taken from nlp-sentiment-analysis
### link to the project: https://github.com/dvdhartsman/NLP-Sentiment-Analysis/blob/main/Text_Classification_Final_Notebook.ipynb

def preprocess(tweet):
    """
    This is a function that is intended to handle all of the tokenization, lemmatization, and other
    preprocessing for our tweet data. It will make use of objects from other libraries, and will return
    a complete list of tokens that are ready to be vectorized into numerical data.
    """
    
    # Create a list of stopwords to be removed from our tokenized word list
    stops = stopwords.words("english")
    # Add punctuation to the list of stopwords
    stops += string.punctuation
    # Providing a regex pattern for the tokenizer to handle
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    # Instantiating a tokenizer
    tokenizer = RegexpTokenizer(pattern)
    # Creating a list of raw tokens
    raw_tokens = tokenizer.tokenize(tweet)
    # Using a comprehension to lower case every token
    lower_tokens = [i.lower() for i in raw_tokens]
    # Remove the stopwords from the list of tokens
    stopped_words = [i for i in lower_tokens if i not in stops]
    
    # Adding parts of speech to prepare for Lemmatization
    
    # This is the initial method to get parts of speech
    stopped_words = pos_tag(stopped_words)
    
    # Get_wordnet_pos() is the function to modify the pos definitions/assignments, creates tuples of (<word>, <pos>)
    stopped_words = [(word[0], get_wordnet_pos(word[1])) for word in stopped_words]
    
    lemmatizer = WordNetLemmatizer() 
    
    # This corrects the parts of speech and maximizes the usefulness of the lemmatization!!!!!
    document = [lemmatizer.lemmatize(word[0], word[1]) for word in stopped_words]
    
    # Re-join the list of cleaned tokens
    cleaned_doc = " ".join(document)
    return cleaned_doc

In [57]:
X = df_train.comment_text
y = df_train[['toxic', 'severe_toxic', 'obscene', 'threat','insult','identity_hate']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify=['toxic', 'severe_toxic','identity_hate'])
y_toxic_train = df_train.toxic
y_severe_toxic_train = df_train.severe_toxic
y_obscene_train = df_train.obscene
y_threat_train = df_train.threat
y_insult_train = df_train['identity_hate']

In [34]:
X_clean = X.apply(preprocess)

In [38]:
count_vec = CountVectorizer()
tf_vec = TfidfVectorizer()

In [39]:
count_vec.fit(X_clean)
tf_vec.fit(X_clean)

TfidfVectorizer()

In [40]:
X_count = count_vec.transform(X_clean)
X_tfidf = tf_vec.transform(X_clean)

In [58]:
X_train_cv, X_test_cv, y_train, y_test = train_test_split(X_count,y, random_state = 42)

In [59]:
X_train_tf, X_test_tf, y_train, y_test = train_test_split(X_tfidf,y, random_state = 42)

In [60]:
X_train_cv

<119678x157975 sparse matrix of type '<class 'numpy.int64'>'
	with 3190601 stored elements in Compressed Sparse Row format>

In [61]:
X_train_tf

<119678x157975 sparse matrix of type '<class 'numpy.float64'>'
	with 3190601 stored elements in Compressed Sparse Row format>

In [62]:
y_train.value_counts()

toxic  severe_toxic  obscene  threat  insult  identity_hate
0      0             0        0       0       0                107512
1      0             0        0       0       0                  4309
                     1        0       1       0                  2833
                                      0       0                  1309
                     0        0       1       0                   893
       1             1        0       1       0                   731
       0             1        0       1       1                   470
0      0             1        0       0       0                   237
                     0        0       1       0                   213
1      1             1        0       1       1                   193
0      0             1        0       1       0                   141
1      1             1        0       0       0                   124
       0             0        0       1       1                   106
                     1        

In [78]:
#X_train.apply(preprocess)

"""id
0000997932d777bf    explanation edits make username hardcore metal...
000103f0d9cfb60f    d'aww match background colour i'm seemingly st...
000113f07ec002fd    hey man i'm really try edit war guy constantly...
0001b41b1c6bb37e    can't make real suggestion improvement wonder ...
0001d958c54c6e35                 sir hero chance remember page that's
                                          ...                        
ffe987279560d7ff    second time ask view completely contradict cov...
ffea4adeee384e90                 ashamed horrible thing put talk page
ffee36eab5c267c9    spitzer umm there actual article prostitution ...
fff125370e4aaaf3    look like actually put speedy first version de...
fff46fc426af1f9a    really think understand come idea bad right aw...
Name: comment_text, Length: 159571, dtype: object"""

id
0000997932d777bf    explanation edits make username hardcore metal...
000103f0d9cfb60f    d'aww match background colour i'm seemingly st...
000113f07ec002fd    hey man i'm really try edit war guy constantly...
0001b41b1c6bb37e    can't make real suggestion improvement wonder ...
0001d958c54c6e35                 sir hero chance remember page that's
                                          ...                        
ffe987279560d7ff    second time ask view completely contradict cov...
ffea4adeee384e90                 ashamed horrible thing put talk page
ffee36eab5c267c9    spitzer umm there actual article prostitution ...
fff125370e4aaaf3    look like actually put speedy first version de...
fff46fc426af1f9a    really think understand come idea bad right aw...
Name: comment_text, Length: 159571, dtype: object

# Modeling

In [76]:
class MultiOutput():
    # a class to more easily test, interpret and store different classifiers for a multioutput dataset
    model_list = []
    model_df = pd.DataFrame(columns = ['Classifier', 'train_accuracy','train_prec','train_recall','train_f1',
                                      'test_accuracy','test_prec','test_recall','test_f1'])
    
    def __init__(self, name, clf, X_train, X_test, y_train, y_test):
        self.name = name
        self.clf = classifier
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        
        # Measuring model
        self.train_results = cross_validate(self.model, self.X_train, self.y_train, scoring=[
            'precision', 'accuracy', 'recall', 'f1', 'neg_log_loss'], n_jobs=4, verbose=1)
        

## Baseline Dummy 

In [63]:
dummy = DummyClassifier(strategy='most_frequent')

In [64]:
dummy_clf = MultiOutputClassifier(dummy).fit(X_train_cv,y_train)

In [65]:
accuracy_score(y_train, dummy_clf.predict(X_train_cv))

0.898343889436655

In [66]:
accuracy_score(y_test, dummy_clf.predict(X_test_cv))

0.8982528263103803

In [67]:
y_train

,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,
c912439967ba8a35,0,0,0,0,0,0
b312f612d3394d5b,0,0,0,0,0,0
813c210bf7f27377,0,0,0,0,0,0
c231999bc75dcd9e,0,0,0,0,0,0
d60a1de68cf593c1,0,0,0,0,0,0
...,...,...,...,...,...,...
811ed72c51830f42,0,0,0,0,0,0
2acc7c7d0386401f,0,0,0,0,0,0
c1f95b89050a9ee4,1,0,0,0,0,0


## Logistic Regression

In [68]:
logreg_clf = MultiOutputClassifier(LogisticRegression()).fit(X_train_cv, y_train)

C:\Users\rchag\anaconda3\envs\learn-env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\rchag\anaconda3\envs\learn-env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

In [96]:
accuracy_score(y_train, logreg_clf.predict(X_train_cv))

0.9374488210030247

In [69]:
accuracy_score(y_test, logreg_clf.predict(X_test_cv))

0.9126663825733838

In [70]:
logreg_clf.predict(X_test_cv)

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [71]:
logreg_clf_tf = MultiOutputClassifier(LogisticRegression()).fit(X_train_tf, y_train)

C:\Users\rchag\anaconda3\envs\learn-env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [98]:
accuracy_score(y_train, logreg_clf_tf.predict(X_train_tf))

0.9245558916425742

In [72]:
accuracy_score(y_test, logreg_clf_tf.predict(X_test_tf))

0.9181560674805104

## Random Forest

In [78]:
from sklearn.ensemble import RandomForestClassifier

rfclf = MultiOutputClassifier(RandomForestClassifier(n_jobs = -1)).fit(X_train_cv, y_train)

In [79]:
accuracy_score(y_test, rfclf.predict(X_test_cv))

0.9126663825733838

In [80]:
rfclf_tf = MultiOutputClassifier(RandomForestClassifier(n_jobs = -1)).fit(X_train_tf, y_train)

In [81]:
accuracy_score(y_test, rfclf_tf.predict(X_test_tf))

0.9142456069987216

## Multinomial Naive Bayes

In [91]:
from sklearn.naive_bayes import MultinomialNB

mnb_cv = MultiOutputClassifier(MultinomialNB()).fit(X_train_cv, y_train)

In [92]:
accuracy_score(y_test, mnb_cv.predict(X_test_cv))

0.8980021557666759

In [94]:
mnb_tf = MultiOutputClassifier(MultinomialNB()).fit(X_train_tf, y_train)

In [95]:
accuracy_score(y_test, mnb_tf.predict(X_test_cv))

0.9013611410523149

## Sequential

In [104]:
from keras.models import Sequential

In [105]:
model = Sequential()

In [ ]:
model.add(Dense(20))